In [21]:
#= Importy i załadowanie danych =#
using JLD2
using Printf
using Statistics
using LinearAlgebra

include("src/AD.jl")
include("src/NN.jl")

Main.NN

In [22]:
#= Przygotowanie danych =#
file = load("data/imdb_dataset_prepared.jld2")
X_train = Int.(file["X_train"])
y_train = vec(Float32.(file["y_train"]))
X_test  = Int.(file["X_test"])
y_test  = vec(Float32.(file["y_test"]))
embeddings = file["embeddings"]
vocab = file["vocab"]
embedding_dim = size(embeddings, 1)

50

In [23]:
dataset = NN.DataLoader((X_train, y_train), batchsize=64, shuffle=true)

Main.NN.DataLoader((Float32[6391.0 286.0 … 11382.0 8311.0; 143.0 286.0 … 4254.0 2891.0; … ; 12849.0 12849.0 … 12849.0 12849.0; 12849.0 12849.0 … 12849.0 12849.0], Float32[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]), 64, true, [19759, 38382, 136, 884, 26637, 29063, 752, 7925, 30625, 35172  …  27258, 5665, 6175, 33858, 34653, 23892, 13704, 36649, 25407, 39177])

In [24]:
model = NN.Chain(
    x -> Int.(x),
    NN.Embedding(length(vocab), embedding_dim, embeddings),
    x -> permutedims(x, (2, 1, 3)),
    NN.Conv1D(3, embedding_dim, 8, AD.relu),
    NN.MaxPool1D(8),
    NN.flatten,
    NN.Dense(128, 1, AD.sigmoid)
)

Main.NN.Chain(Any[var"#13#15"(), Main.NN.Embedding(12849, 50, Float32[0.90951 -0.58014 … 0.014323 0.0; -0.20702 -1.1316 … -0.74624 0.0; … ; -0.60396 0.1722 … -0.079873 0.0; -0.31136 1.2622 … -0.89164 0.0], Int64[], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]), var"#14#16"(), Main.NN.Conv1D(3, 50, 8, Float32[-0.17211856 -0.0064253253 … -0.05562423 -0.08469726; -0.08166572 0.034175422 … 0.22911519 -0.06999489; -0.01276053 0.053518537 … -0.0454837 0.14835237;;; 0.0054890173 0.22387534 … -0.15616374 -0.23324706; -0.07497164 0.18194051 … 0.19667698 -0.019543901; 0.03380009 0.042077277 … 0.041229825 0.06587537;;; -0.06079337 0.12012959 … 0.06387137 0.021520622; 0.095071785 0.10076227 … 0.13150492 -0.015644703; -0.089909546 -0.2149365 … -0.011547326 0.07722337;;; -0.029296499 0.013090888 … 0.106293894 0.09276467; -0.03304062 0.17795461 … -0.025395023 -0.0041076024; 0.12903029 0.06062223 … 0.038451802 0.25741935;;; -0.10302168 0.032499105 … 0.0094989

In [25]:
accuracy(m, x, y) = mean((vec(m(x)) .> 0.5) .== (y .> 0.5))

accuracy (generic function with 1 method)

In [26]:
#= Trening modelu =#
opt = NN.Adam(Float32(0.001))
epochs = 5
NN.train_model(model, dataset, X_test, y_test, opt, epochs)

Epoch: 1 (21.41s) 	Train: (l: 0.59, a: 0.68) 	Test: (l: 0.50, a: 0.76) 	Memory allocated: 23704.395 MB
Epoch: 2 (21.40s) 	Train: (l: 0.48, a: 0.77) 	Test: (l: 0.46, a: 0.79) 	Memory allocated: 23691.986 MB
Epoch: 3 (21.28s) 	Train: (l: 0.45, a: 0.79) 	Test: (l: 0.44, a: 0.80) 	Memory allocated: 23700.028 MB
Epoch: 4 (21.15s) 	Train: (l: 0.43, a: 0.80) 	Test: (l: 0.42, a: 0.81) 	Memory allocated: 23704.314 MB
Epoch: 5 (21.58s) 	Train: (l: 0.40, a: 0.82) 	Test: (l: 0.40, a: 0.82) 	Memory allocated: 23707.371 MB
